In [14]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal, arange
from scipy.signal import hilbert, find_peaks_cwt
import matplotlib.pyplot as plt
import matplotlib.axes as matax
import matplotlib.mlab as mlab
import sys
from math import exp
import operator

In [5]:
sampFreq7, unit7sampArray = read('unit7wav5.wav');
sampFreq8, unit8sampArray = read('unit8wav5.wav');
sampFreq13, unit13sampArray = read('unit13wav5.wav');
sampFreq20, unit20sampArray = read('unit20wav5.wav');

In [28]:
#sample frequency set equal to one of the sound files
#sound frequency since it should be the same for all
#files anyway
sampFreq = sampFreq7

#start time and duration measured in seconds
startTime = 0;
duration = duration = int(len(unit7sampArray)/sampFreq);

#in seconds
deadzoneInSeconds = 0.01

###DO NOT CHANGE THE NEXT TWO VARIABLES!!! 
#IF CHANGED PLEASE RESET leftOffset TO 50 AND rightOffset to 200
leftOffsetClassif = 50;  # how many frames we should expand the interval to the left (for the classfication algorithm)
rightOffsetClassif = 200; # how many frames we should expand ther interval to the right (for the classificaiton algorithm)
###DO NOT CHANGE THE LAST TWO VARIABLES!!!

#The deltaT (in seconds) for finding the same snap, for now I'll set it as some number that's 
#somewhat reasonable? idk....
deltaTinSeconds = 0.25;

#actual array indexes for splicing
startIndex = sampFreq7 * startTime;
endIndex = startIndex + sampFreq * duration;


unit7splicedArray = unit7sampArray[startIndex:endIndex];
unit8splicedArray = unit8sampArray[startIndex:endIndex];
unit13splicedArray = unit13sampArray[startIndex:endIndex];
unit20splicedArray = unit20sampArray[startIndex:endIndex];

In [20]:
#preparing 2 time arrays for the spliced signals for plotting (if needed) the x axis. 
#Indexes and seconds.
timeArrayInIndexes = np.arange(0, endIndex-startIndex, 1)
timeArrayInSeconds = timeArrayInIndexes / sampFreq

In [21]:
#takes in the sliced array and 
#returns an array of indexes and seconds where the amplitude
#is above the threshold. Already takes into account
#the small deadzone
def detectAboveThreshold(splicedArray):
    #setting up the amplitude threshold
    ampThreshold = np.std(splicedArray) * 10;
    
    #creates a boolean array for each index of the spiced sound array
    #each index of this boolean array corresponds to 1 index of the 
    #spliced sound array. The value will be true if above threshold,
    #false otherwise
    aboveThres = splicedArray > ampThreshold 
    
    #pick out the time when the amplitude is above threshold
    # current = the first time in seconds frame has amplitude above threshold
    # highAmplitudeSeconds = the time where each loud signal above threshold
    # highAmplitudeIndexes = the time frame indices of each loud signal above threshold
    highAmplitudeSeconds = [];
    highAmplitudeIndexes = [];
    current = timeArrayInSeconds[aboveThres][0] 
    
    #initializing the arrays with the first values before the loop 
    highAmplitudeSeconds.append(current)
    highAmplitudeIndexes.append(current * sampFreq)
    
    # set a deadzone (set to 0.01 seconds in the earlier cell) 
    # time window that slides across the current array
    # so that we only collect one time data for each loud signal
    for timeInSeconds in timeArrayInSeconds[aboveThres]: 
    
        if (timeInSeconds > current + deadzoneInSeconds ):
            highAmplitudeIndexes.append(timeInSeconds * sampFreq)
            highAmplitudeSeconds.append(timeInSeconds)
            current = timeInSeconds
            
    
    return highAmplitudeIndexes, highAmplitudeSeconds;
    

In [22]:
#grabs the indexes and times when amplitude goes above threshold
highAmpIndexesUnit7, highAmpSecondsUnit7 = detectAboveThreshold(unit7splicedArray)
highAmpIndexesUnit8, highAmpSecondsUnit8 = detectAboveThreshold(unit8splicedArray)
highAmpIndexesUnit13, highAmpSecondsUnit13 = detectAboveThreshold(unit13splicedArray)
highAmpIndexesUnit20, highAmpSecondsUnit20 = detectAboveThreshold(unit20splicedArray)

In [30]:
#a function that takes in the array of indexes of all signals (both snaps and pings)
#above threshold and outputs the array of indexes where SNAPS were found. 
def getSnapIndexes(splicedArray, highAmpIndexes):
    snapIndexes = [];
    
    for i in range(0, len(highAmpIndexes)):
        frontIndex = int(highAmpIndexes[i] - leftOffsetClassif);
        backIndex = int(highAmpIndexes[i] + rightOffsetClassif);
        
        signalForClassification = splicedArray[frontIndex:backIndex]
        classification = classify(signalForClassification)
        
        
        if(classification == "Snap"):
            snapIndexes.append(highAmpIndexes[i]);
        
    
    return snapIndexes;
            

In [31]:
def getFrequencies(signal):
    length = len(signal) # length of the signal
    #k = arange(length)
    #T = length/sampFreq
    #frq = k/T # two sides frequency range
    #frq = frq[range(np.int(length/2))] # one side frequency range

    frequencies = np.fft.fft(signal)/length # fft computing and normalization
    frequencies = frequencies[range(np.int(length/2))]
    
    return abs(frequencies)

In [32]:
#I did not change this snippet of code, it should be identical to the other one
def getRatio(signal):
    frequencies = getFrequencies(signal)
    ratio = sum(frequencies[34:68] * 100/sum(frequencies))
    return ratio;
    

In [33]:
#I did not change this snippet of code, it should be identical to the other one
def classify(signal):
    #print(len(signal))
    
    ratio = getRatio(signal)
    #print(ratio)
    if (ratio > 70):
        return "Ping"
    
    elif (ratio < 50):
        return "Snap"
    else:
        return "Undefined"

In [34]:
snapIndexesUnit7 = getSnapIndexes(unit7splicedArray, highAmpIndexesUnit7);
snapIndexesUnit8 = getSnapIndexes(unit8splicedArray, highAmpIndexesUnit8);
snapIndexesUnit13 = getSnapIndexes(unit13splicedArray, highAmpIndexesUnit13);
snapIndexesUnit20 = getSnapIndexes(unit20splicedArray, highAmpIndexesUnit20);

In [39]:
#test
print(len(snapIndexesUnit20))

print(len(highAmpIndexesUnit20))


948
1733


In [ ]:
#TODO: function that would look at the snapIndexes of the 4 files and 
#group the snaps